In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import sqlite3

from dateutil.parser import parse

In [2]:
folderpath = './varfind/'
dat_filename = 'varfind.dat'
img_filename = 'varfind.fts'
indata = np.loadtxt(folderpath + dat_filename, delimiter=" ")

image_file = fits.open(folderpath+img_filename)
image_data = image_file[0].data
image_header = image_file[0].header

JD = image_header["JD"]
exposure = image_header["EXPTIME"]
filter_type = image_header["FILTER"]
date = parse(image_header["DATE-OBS"]).date()

#Creating Arrays for Data
JD = indata[:, 0]
mag_JD = indata[:, 1::2]
mag_star = np.transpose(mag_JD)
magerror_JD = indata[:, ::2]
magerror_JD = magerror_JD[:, 1:]

image_file.close()

In [3]:
#Saving FITS Image
fig = plt.figure(figsize=(12,12))

plt.imshow(image_data, cmap='gray')
plt.colorbar()

fig.suptitle('NGC 2180 Star Field | Exposure: ' + str(exposure) + '| Filter: ' + filter_type, fontsize=20)

fig.savefig('./lightcurves/'+'field' + str(date) + '.png')

plt.close()

In [4]:
#READ XML FILE
#TRANSLATE XY COORDS TO RA DEC
#FIND PERIODS OF LCs
#DB setup
#RNNs and Feature Extraction
#Catalina
#PULL DATA AND COMPARE LCs

In [5]:
#If a comp star does not have a data point, do you remove all data points for all comp stars or negate that star from the average for that point
def defCompStars(stars):
    comp_mag = [mag_star[s] for s in stars]
    comp_JD = JD.tolist()
    '''for s in stars:
        comp_mag = np.add(comp_mag, mag_star[s])
        
    comp_mag /= len(stars)
    #Is this step necessary
    comp_mag = np.ndarray.tolist(comp_mag)
    
    #Popping Comp Star Values where a comp is null
    for idx, val in enumerate(reversed(comp_mag)):
        del_el = False
        index = (len(mag_star[0]) - 1) - idx
        for n in stars:
            del_el = del_el or (mag_star[n][index] >= 99)
        if del_el:
            print(index)
            comp_mag.pop(index)
            comp_JD.pop(index)
    '''
    return comp_mag, comp_JD

In [6]:
c_mag, c_JD = defCompStars([1, 2, 500])
print(len(c_mag[1]))

100


In [7]:
#Plotting the nth star's light curve
def plotLightCurve(n):
    
    mag = mag_star[n].tolist()
    nJD = JD.tolist()
    for idx, val in enumerate(reversed(mag)):
        if(val >= 99):
            mag.pop((len(mag_star[n]) - 1) - idx)
            nJD.pop((len(mag_star[n]) - 1) - idx)

    fig = plt.figure(figsize=(20,8))
    plt.scatter(nJD, mag)
    plt.xlabel(r'${\rm JD}$', fontsize=18)
    plt.ylabel(r'${\rm Mag}$', fontsize=18)
    plt.xlim(np.min(nJD)-0.01, np.max(nJD)+0.01)
    plt.ylim((np.min(mag)-0.3, np.max(mag)+0.3))
    fig.suptitle('NGC 2180: Star ' + str(n) + '| Date: ' + str(date), fontsize=18)
    fig.savefig('./lightcurves/'+'lc_' + str(n) + '.png')
    plt.close()